In [38]:
import json
import random

class Attaque:
    def __init__(self, nom, degats, precision):
        self.nom = nom
        self.degats = degats
        self.precision = precision

class Navire:
    def __init__(self, type_navire, sante_base, esquive_base, taille, initiative, attaques):
        self.type_navire = type_navire
        self.sante_base = sante_base
        self.esquive_base = esquive_base
        self.taille = taille
        self.initiative = initiative
        self.attaques = [Attaque(att.nom, att.degats, att.precision) for att in attaques]
        self.destructions = 0
        self.sante = sante_base
        self.total_degats_infliges = 0
        self.active = True

    def calculer_stats_effectives(self, moral, discipline, bonus_amiral):
        self.sante = self.sante_base * (1 + moral / 100)
        self.esquive = self.esquive_base + bonus_amiral['esquive']
        self.initiative += bonus_amiral['initiative']
        for attaque in self.attaques:
            attaque.degats = attaque.degats * (1 + discipline / 100) + bonus_amiral['degats']
            attaque.precision = attaque.precision + bonus_amiral['precision']

    def est_destruit(self):
        return self.sante <= 0

    def etat(self):
        if self.sante <= 0:
            return 'détruit'
        elif self.sante <= self.sante_base * 0.5:
            return 'endommagé'
        else:
            return 'opérationnel'

def choisir_cible(taille_cible, defenders_active):
    distances = [(abs(nav.taille - taille_cible), nav) for nav in defenders_active]
    min_distance = min(distances, key=lambda x: x[0])[0]
    possibles_cibles = [nav for dist, nav in distances if dist == min_distance]
    return random.choice(possibles_cibles)

def attaque_touche(precision_attaque, esquive_defenseur):
    # Première étape : Vérification si l'attaque atteint la zone de la cible
    if random.uniform(0, 100) <= precision_attaque:
        # Deuxième étape : Vérification si la cible esquive l'attaque
        if random.uniform(0, 100) > esquive_defenseur:
            return True  # L'attaque touche effectivement
    return False  # L'attaque manque ou est esquivée


def simulate_battle(flotte1, flotte2, admiral1, admiral2, max_turns=30):
    turn_count = 0
    while any(nav.active for nav in flotte1) and any(nav.active for nav in flotte2) and turn_count < max_turns:
        tous_navires = sorted((nav for nav in flotte1 + flotte2 if nav.active), key=lambda nav: nav.initiative, reverse=True)
        for navire in tous_navires:
            flotte_attacking, flotte_defending, admiral_attacking = (flotte1, flotte2, admiral1) if navire in flotte1 else (flotte2, flotte1, admiral2)
            if any(nav.active for nav in flotte_defending):
                result_dice = random.randint(1, 10)  # Lancer un dé à 10 faces
                defender = choisir_cible(result_dice, [nav for nav in flotte_defending if nav.active])
                for attaque in navire.attaques:
                    if attaque_touche(attaque.precision, defender.esquive):
                        damage = attaque.degats
                        defender.sante -= damage
                        navire.total_degats_infliges += damage
                        if defender.est_destruit():
                            defender.active = False
                            navire.destructions += 1
            else:
                break
        turn_count += 1
    winner = "Flotte 1" if any(nav.active for nav in flotte1) else "Flotte 2"
    return turn_count, winner

def charger_flotte(fichier_json):
    flotte = []
    initial_count = {}
    with open(fichier_json, 'r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data['navires']:
            attaques = [Attaque(att['nom'], att['degats'], att['precision']) for att in item['attaques']]
            for _ in range(item['quantite']):
                navire = Navire(item['type'], item['sante_base'], item['esquive_base'], item['taille'], item['initiative'], attaques)
                flotte.append(navire)
            initial_count[item['type']] = initial_count.get(item['type'], 0) + item['quantite']
    return flotte, initial_count

def afficher_resultats(flotte, initial_count):
    resultat = {key: {'opérationnel': 0, 'endommagé': 0, 'détruit': 0, 'destructions': 0, 'total_degats': 0} for key in initial_count}
    for navire in flotte:
        key = navire.type_navire
        if navire.active:
            etat = navire.etat()
            if etat == 'opérationnel':
                resultat[key]['opérationnel'] += 1
            elif etat == 'endommagé':
                resultat[key]['endommagé'] += 1
        else:
            resultat[key]['détruit'] += 1
        resultat[key]['destructions'] += navire.destructions
        resultat[key]['total_degats'] += navire.total_degats_infliges
    for type_navire, stats in resultat.items():
        print(f"{type_navire}: Opérationnels - {stats['opérationnel']}, Endommagés - {stats['endommagé']}, Détruits - {stats['détruit']}, Destructions - {stats['destructions']}, Dégâts infligés - {stats['total_degats']}")

def main():
    flotte1, initial_count1 = charger_flotte("flotte1.json")
    flotte2, initial_count2 = charger_flotte("flotte2.json")
    admiral1 = {'degats': 5, 'precision': 10, 'esquive': 5, 'initiative': 20}
    admiral2 = {'degats': 10, 'precision': 5, 'esquive': 10, 'initiative': 15}
    for navire in flotte1:
        navire.calculer_stats_effectives(10, 15, admiral1)
    for navire in flotte2:
        navire.calculer_stats_effectives(5, 20, admiral2)
    tours, gagnant = simulate_battle(flotte1, flotte2, admiral1, admiral2)
    print("Résultats de la bataille pour Flotte 1:")
    afficher_resultats(flotte1, initial_count1)
    print("Résultats de la bataille pour Flotte 2:")
    afficher_resultats(flotte2, initial_count2)
    print(f"Nombre de tours: {tours}")
    print(f"Gagnant: {gagnant}")

if __name__ == "__main__":
    main()



Résultats de la bataille pour Flotte 1:
Corvette: Opérationnels - 1, Endommagés - 1, Détruits - 8, Destructions - 0, Dégâts infligés - 2330.5
Destroyer: Opérationnels - 0, Endommagés - 0, Détruits - 5, Destructions - 0, Dégâts infligés - 582.0
Chasseur: Opérationnels - 14, Endommagés - 0, Détruits - 16, Destructions - 12, Dégâts infligés - 6860.0
Résultats de la bataille pour Flotte 2:
Croiseur: Opérationnels - 0, Endommagés - 0, Détruits - 3, Destructions - 14, Dégâts infligés - 5010.0
Cuirassé: Opérationnels - 0, Endommagés - 0, Détruits - 2, Destructions - 4, Dégâts infligés - 3640.0
Fregate: Opérationnels - 0, Endommagés - 0, Détruits - 7, Destructions - 13, Dégâts infligés - 1680.0
Nombre de tours: 17
Gagnant: Flotte 1
